# INICIALIZACIÓN

In [ ]:
# @title INSTALACIÓN DE PAQUETES

# Verificar e instalar el paquetes
try:
    import langchain
except ImportError:
    print("El paquete 'langchain' no está instalado. Procediendo con la instalación...")
    !pip install --upgrade langchain langchain_community openai

try:
    import langchain_community
except ImportError:
    print("El paquete 'langchain_community' no está instalado. Procediendo con la instalación...")
    !pip install --upgrade langchain_community

try:
    import openai
except ImportError:
    print("El paquete 'openai' no está instalado. Procediendo con la instalación...")
    !pip install --upgrade openai

In [ ]:
# @title IMPORTACIÓN DE MÓDULOS
import gspread
from google.colab import auth
from google.auth import default

from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

# DECLARACIÓN DE FUNCIONES

In [ ]:
# @title summarize_transcriptions_to_sheet()
def summarize_transcriptions_to_sheet(params: dict) -> None:
    """
    Resume transcripciones de reuniones almacenadas en Google Sheets y guarda los resúmenes en otro Google Sheet.

    Args:
        params (dict):
            - input_sheet_id (str): ID del Google Sheet que contiene las transcripciones.
            - output_sheet_id (str): ID del Google Sheet donde se guardarán los resúmenes.
            - system_prompt (str): Prompt para inicializar el contexto del sistema en el LLM.
            - OpenAI_API_key (str): Clave de la API de OpenAI.
            - model_name (str): Nombre del modelo de OpenAI a utilizar.
            - row_range (str): Rango de filas a procesar, por ejemplo "1-10", "all" o "1".
            - temperature (float): Valor de temperatura para el modelo LLM.
            - ConversationBufferMemory_params (dict): Configuración para ConversationBufferMemory.
    Returns:
        None
    Raises:
        ValueError: Si faltan parámetros esenciales en el diccionario.
    """
    import time
    from datetime import datetime

    # Validar parámetros
    input_sheet_id = params.get("input_sheet_id")
    output_sheet_id = params.get("output_sheet_id")
    system_prompt = params.get("system_prompt")
    OpenAI_API_key = params.get("OpenAI_API_key")
    model_name = params.get("model_name", "gpt-4")
    row_range = params.get("row_range", "all")
    temperature = params.get("temperature", 0.7)
    ConversationBufferMemory_params = params.get("ConversationBufferMemory_params", {})

    if not all([input_sheet_id, output_sheet_id, system_prompt, OpenAI_API_key]):
        raise ValueError("Faltan parámetros esenciales en 'params'.")

    # Autenticación de Google Sheets
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Abrir Google Sheet con transcripciones
    sheet = gc.open_by_key(input_sheet_id).sheet1
    all_data = sheet.get_all_records()

    if not all_data:
        raise ValueError("El Google Sheet no contiene datos para procesar.")

    # Filtrar datos según el rango de filas
    if row_range.lower() == "all":
        data = all_data
    elif "-" in row_range:
        start_row, end_row = map(int, row_range.split("-"))
        data = all_data[start_row - 1:end_row]
    else:
        data = [all_data[int(row_range) - 1]]

    # Configurar el modelo de OpenAI
    llm = ChatOpenAI(
        api_key=OpenAI_API_key,
        model_name=model_name,
        temperature=temperature
    )

    # Configuración de prompts
    system_message = SystemMessagePromptTemplate.from_template(system_prompt)
    human_message = HumanMessagePromptTemplate.from_template(
        """Aquí tienes la transcripción (texto completo):
        {transcription}"""
    )
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # Crear memoria y la cadena LLM
    memory = ConversationBufferMemory(**ConversationBufferMemory_params)
    chain = LLMChain(llm=llm, prompt=chat_prompt, memory=memory)

    results = []

    for row in data:
        start_time = time.time()
        file_name = row.get("file_name", "Archivo desconocido")
        print(f"\n[INFO] Procesando archivo: {file_name}")

        # Combinar todas las partes de la transcripción
        transcription_parts = [row.get(f"transcription_part_{i}", "") for i in range(1, 11)]
        full_transcription = " ".join(filter(None, transcription_parts))

        if full_transcription.strip():
            print(f"[INFO] Longitud de la transcripción: {len(full_transcription)} caracteres")
            # Generar el resumen
            response = chain.run(transcription=full_transcription)
            summary_text = response.strip()

            process_duration = time.time() - start_time
            print(f"[SUCCESS] Resumen generado en {process_duration:.2f} segundos:")
            print(f"[SUMMARY] {summary_text}")

            results.append({
                "file_name": file_name,
                "summary": summary_text
            })
        else:
            print(f"[WARNING] Transcripción vacía en el archivo: {file_name}")

    # Guardar resúmenes en Google Sheets
    result_sheet = gc.open_by_key(output_sheet_id).sheet1
    result_sheet.append_row(["file_name", "summary"])
    for res in results:
        result_sheet.append_row([res["file_name"], res["summary"]])

    print("\n[INFO] Resúmenes generados y guardados con éxito.")


# DEFINICIÓN DE PROMPTS

In [ ]:
# @title system_prompt_reuniones_general

system_prompt_reuniones_general = """
Eres un analista experto en transcripciones técnicas relacionadas con implementaciones de ERP, como Microsoft Dynamics 365 Business Central, integraciones con sistemas adicionales (como SolyMat) y procesos internos de negocio. Tu tarea es elaborar un **resumen detallado** y estructurado de la conversación, sin omitir ningún aspecto relevante.

**Características del Resumen:**

1. **Extremo Nivel de Detalle:** Captura cada tema, subtema y aspecto específico discutido, proporcionando ejemplos, aclaraciones, y contexto cuando sea necesario.
2. **Estructuración Granular:** Divide la información en secciones y subsecciones jerárquicas. Usa listas y párrafos explicativos para desglosar cada punto.
3. **Corrección e Interpretación:** Interpreta términos técnicos o nombres mal escritos, aclarando el significado o contexto cuando sea necesario. Si hay ambigüedades, inclúyelas con comentarios explicativos.
4. **Puntos Clave y Decisiones:** Resalta explícitamente los acuerdos alcanzados, las dudas pendientes, y las tareas a realizar.
5. **Aclaraciones Técnicas:** Explica cómo funcionan los sistemas mencionados o cómo interactúan entre ellos, basándote en la información de la transcripción.
6. **Exclusión de Ruido:** Ignora frases o comentarios irrelevantes (chistes, interrupciones, repeticiones) que no aporten valor técnico o informativo.
>
**Formato de Salida Esperado:**

### Resumen Ejecutivo
Una breve descripción de los temas principales tratados en la conversación.

### Resumen Detallado
#### 1. Tema Principal
- **Subtema 1:** Explicación detallada con ejemplos y aclaraciones.
- **Subtema 2:** Información granular sobre el tema, con todas las decisiones y propuestas.

#### 2. Tema Principal
- **Subtema 1:** ...
- **Subtema 2:** ...

### Decisiones y Acciones Pendientes
- **Decisión 1:** Descripción clara de la decisión tomada.
- **Tarea Pendiente 1:** Acción específica asignada a un responsable o equipo.

**Requisitos Adicionales:**
- **Contexto Amplio:** Si un término o proceso requiere explicación adicional para ser comprensible, inclúyela.
- **Claridad en las Decisiones:** Deja claro quién toma decisiones y qué impacto tendrán en el sistema o el flujo de trabajo.
- **Referencias Cruzadas:** Si un tema discutido se relaciona con otro mencionado anteriormente, inclúyelo para evitar duplicaciones y facilitar la comprensión.

"""




In [ ]:
# @title system_prompt_reuniones_ERP

system_prompt_reuniones_ERP = """
**Aspectos Específicos a Capturar:**
- Terminología clave (e.g., "módulo", "convocatoria", "programa formativo") con definiciones claras.
- Estructura de procesos internos: desde la matriculación hasta la facturación y seguimiento.
- Detalles de las configuraciones discutidas, incluyendo opciones específicas y justificaciones (e.g., formas de pago, números de serie).
- Problemas detectados y cómo se propone solucionarlos.
- Interacciones entre los sistemas mencionados y el flujo de información entre ellos.

**Términos mal transcritos:**
Si ten encuentras estos términos mal transcritos, corrígelos:
Haspo, JASPOP es HubSpot
Solimat es SolyMat
píanel es P&L
"""

system_prompt = system_prompt_reuniones_general + system_prompt_reuniones_ERP

# EJECUCIÓN DE RESÚMENES

In [ ]:
# @title Configuración

from google.colab import userdata
OpenAI_API_key = userdata.get('OpenAI_API_key')

return_messages = False             #texto concatenado en lugar de objetos de mensaje @param ["False","True"]
human_prefix = "Usuario"            #@param {type:"string"}
ai_prefix = "Asistente"             #@param {type:"string"}
memory_key = "history"              #@param {type:"string"}

input_sheet_id = "18esoAyRzemkA3GLVTDV6k45U5KqQzkrULYt3sE99KiQ" #@param {type:"string"}
output_sheet_id = "1g0ALYqLn4kNVNiXuJJG5Zcv8UO7HU1WiF2xGL2FQfp8" #@param {type:"string"}
system_prompt = system_prompt
model_name = "chatgpt-4o-latest" #@param ["gpt-4", "gpt-4-turbo", "chatgpt-4o-latest", "gpt-4o", "gpt-4o-mini", "o1", "o1-mini-2024-09-12"]
# https://platform.openai.com/docs/models#current-model-aliases
temperature = 0.7 #@param {type:"number"}

ConversationBufferMemory_params = {
    "return_messages": return_messages,
    "human_prefix": human_prefix,
    "ai_prefix": ai_prefix,
    "memory_key": memory_key
}

row_range = "all" #@param ["all", "1", "1-10", "11-20"]
params = {
    "input_sheet_id": input_sheet_id,
    "output_sheet_id": output_sheet_id,
    "system_prompt": system_prompt,
    "OpenAI_API_key": OpenAI_API_key,
    "model_name": model_name,
    "row_range": row_range,
    "temperature": temperature,
    "ConversationBufferMemory_params": ConversationBufferMemory_params
}

# Ejecutar la función
summarize_transcriptions_to_sheet(params)


[INFO] Procesando archivo: 01 - Ventas y Cobros-20240604_100548 - v1.0 20240604.mp4
[INFO] Longitud de la transcripción: 87175 caracteres
[SUCCESS] Resumen generado en 29.39 segundos:
[SUMMARY] ### Resumen Ejecutivo  
La conversación ha girado en torno a la explicación detallada de los procesos, flujos de información y estructuras organizativas de la empresa en relación con la implementación de Microsoft Dynamics 365 Business Central y su integración con otros sistemas como Solimat y HubSpot. Se han discutido aspectos clave de nomenclatura, estructuras financieras, configuraciones técnicas y operativas, y se han planteado dudas y reflexiones para ajustar los procesos internos a las funcionalidades del nuevo ERP.

---

### Resumen Detallado  

#### 1. Introducción a la Nomenclatura y Estructura Organizativa  
- **Estructuración Académica:**  
  - Los programas formativos son el producto principal que se vende al cliente, pero internamente la empresa opera con "módulos" como unidad míni